**Importing Libraries**

In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
!pip install scikit-learn==1.5.2 #For Optimizations

Uploading file

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving QG_jets_1.npz to QG_jets_1 (1).npz


Loading File

In [ ]:
npz_file = np.load("QG_jets_1.npz")

Cheaking The File

In [ ]:
npz_file

NpzFile 'QG_jets_1.npz' with keys: X, y

Converting to 2-D Array and Dataframe

In [ ]:
array_3d = npz_file['X']

In [ ]:
array_3d.shape

(100000, 134, 4)

*AI Code Snippet Used*

In [ ]:
# Flatten the 3D array into 2D (combine the 2nd and 3rd dimensions)
array_2d = array_3d.reshape(-1, array_3d.shape[-1])

# Convert the 2D array into a DataFrame
df = pd.DataFrame(array_2d)

# Show the DataFrame
df.head()

,0,1,2,3
0,0.621580,-0.849014,5.021331,-211.0
1,0.641752,-1.250724,5.440500,211.0
2,0.201318,-0.903872,4.903188,22.0
3,0.212268,-1.530076,5.257723,-211.0
4,0.115716,-1.216379,4.722940,22.0


In [ ]:
y=npz_file['y']
label= pd.DataFrame(y)

In [ ]:
label.head()

,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


Reshaping array to accomodate all particles.

In [ ]:
X=array_3d.reshape(100000, 134 * 4)

In [ ]:
X=pd.DataFrame(X)

In [ ]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,526,527,528,529,530,531,532,533,534,535
0,0.621580,-0.849014,5.021331,-211.0,0.641752,-1.250724,5.440500,211.0,0.201318,-0.903872,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.356561,1.284506,1.746901,22.0,0.164377,1.341791,1.623187,22.0,4.297152,1.000135,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10.850090,1.057670,6.353887,-211.0,0.041348,0.746949,6.138046,22.0,0.516706,1.142490,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.365157,1.008201,1.648735,2112.0,1.096304,1.498324,1.668980,22.0,0.078276,1.519283,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.068548,0.220046,5.998914,211.0,2.667602,0.625267,6.057339,22.0,0.818155,0.563799,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Counting non zero entries in a row.

In [ ]:
non_zero_counts= X.apply(lambda row: np.count_nonzero(row), axis=1)
non_zero_counts

,0
0,296
1,264
2,292
3,328
4,208
...,...
99995,144
99996,152
99997,60
99998,260


In [ ]:
non_zero_counts.describe()

,0
count,100000.000000
mean,172.984000
std,70.453757
min,8.000000
25%,120.000000
50%,164.000000
75%,220.000000
max,536.000000


Taking Mean as number of particles to remove padded values

In [ ]:
X_subset = X.iloc[:, :172]

Implementing XGBOOST without OPTIMIZATIONS(*AI SNIPPET*)

In [ ]:
# Assume X_reshaped and y are already defined
X_train, X_test, y_train, y_test = train_test_split(X_subset, label, test_size=0.3, random_state=42)

# Initialize and train the XGBoost classifier
clf = xgb.XGBClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict on test set
y_pred = clf.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7795333333333333


Trying using full set of data to improve accuracy

In [ ]:
# Assume X_reshaped and y are already defined
X_train, X_test, y_train, y_test = train_test_split(X, label, test_size=0.3, random_state=42)

# Initialize and train the XGBoost classifier
clf = xgb.XGBClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict on test set
y_pred = clf.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7837


Using RandomizedSearchCV to Optimize

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:

clf = xgb.XGBClassifier(random_state=42)

param_grid = {
    'n_estimators': [50, 100, 200],  # Number of boosting rounds
    'max_depth': [3, 6, 10],         # Maximum depth of a tree
    'learning_rate': [0.01, 0.05, 0.1],  # Learning rate (step size)
    'subsample': [0.7, 0.8, 0.9],    # Fraction of samples used for training each tree
    'colsample_bytree': [0.7, 0.8, 0.9],  # Fraction of features used for each tree
    'gamma': [0, 0.1, 0.2]           # Minimum loss reduction to make a further partition
}
#Only Params Grid taken from CHAT_GPT

random_search = RandomizedSearchCV(estimator=clf, param_distributions=param_grid,
                                   scoring='accuracy', cv=5, n_jobs=-1, verbose=1,
                                   n_iter=10, random_state=42)
random_search.fit(X_train, y_train)
print("Best hyperparameters:", random_search.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best hyperparameters: {'subsample': 0.8, 'n_estimators': 200, 'max_depth': 10, 'learning_rate': 0.05, 'gamma': 0.1, 'colsample_bytree': 0.7}


Making scikit-learn compatible with xgboost

In [ ]:
# Initialize and train the XGBoost classifier
clf_opt = xgb.XGBClassifier(n_estimators=200, random_state=42,subsample=0.8,max_depth=10,learning_rate=0.05,gamma=0.1,colsample_bytree=0.7)
clf_opt.fit(X_train, y_train)

# Predict on test set
y_pred = clf_opt.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7886333333333333
